## Recorde the different mouvement for tictactoe 2021

In [ ]:
from reachy_sdk import ReachySDK
import numpy as np
import time 
from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

In [ ]:
reachy = ReachySDK('localhost')

To test the mouvement with your configuration : 

For simple mouvement :

In [ ]:
def goto_position(path): 
    reachy.turn_on('reachy')
    move = np.load(path)
    move.allow_pickle=1
    print(move['move'])
    listMoves = move['move'].tolist()
    listTraj = {}
    #listTraj = { eval('self.'+key):float(val) for key,val in listMoves.items() }
    for key,val in listMoves.items():
        listTraj[eval(key)] = float(val)

    goto(
        goal_positions=listTraj, 
        duration=2.0,
        interpolation_mode=InterpolationMode.MINIMUM_JERK
    )

In [ ]:
grab_index=1
path='/home/reachy/dev/tictactoe2021/reachy_tictactoe/moves-2021_nemo/back_1_upright.npz'
goto_position(path)

For complexe trajectories : 

In [ ]:
def trajectoryPlayer(path):
    reachy.turn_on('reachy')
    move = np.load(path)
    move.allow_pickle=1
    #print(list(move.keys()))
    #print(move['move'].shape)
    listMoves = move['move'].tolist()
    #print(move['move'])
    #print(listMoves)
    listTraj = [val for key,val in listMoves.items()]
    listTraj = np.array(listTraj).T.tolist()

    sampling_frequency = 100  #en hertz

    recorded_joints = []
    for joint,val in listMoves.items():
        if 'neck' in joint : 
            fullName = joint
        elif 'r_' in joint: 
            fullName = joint
        elif 'l_' in joint: 
            fullName = joint
        recorded_joints.append(eval(fullName))
        
    first_point = dict(zip(recorded_joints, listTraj[0]))
    goto(first_point, duration=3.0)

    for joints_positions in listTraj:
        print(joints_positions)
        for joint, pos in zip(recorded_joints, joints_positions):
            joint.goal_position = pos
        time.sleep(1 / sampling_frequency)

In [ ]:
path = '/home/reachy/dev/tictactoe2021/reachy_tictactoe/moves-2021_nemo/put_5.npz'
trajectoryPlayer(path)

In [ ]:
path = '/home/reachy/dev/tictactoe2021/reachy_tictactoe/moves-2021_nemo/put_5.npz'
trajectoryPlayer(path)

Record a new mouvement : 

Bras gauche 

In [ ]:
recorded_joints = [
    reachy.l_arm.l_shoulder_pitch,
    reachy.l_arm.l_shoulder_roll,
    reachy.l_arm.l_arm_yaw,
    reachy.l_arm.l_elbow_pitch,
    reachy.l_arm.l_forearm_yaw,
    reachy.l_arm.l_wrist_pitch,
    reachy.l_arm.l_wrist_roll,
]
        
sampling_frequency = 100  #en hertz
record_duration = 5  #en seconde

Bras droit

In [ ]:
recorded_joints = [
    reachy.r_arm.r_shoulder_pitch,
    reachy.r_arm.r_shoulder_roll,
    reachy.r_arm.r_arm_yaw,
    reachy.r_arm.r_elbow_pitch,
    reachy.r_arm.r_forearm_yaw,
    reachy.r_arm.r_wrist_pitch,
    reachy.r_arm.r_wrist_roll,
]
        
sampling_frequency = 100  #en hertz
record_duration = 5  #en seconde

In [ ]:
reachy.turn_on('reachy')
goto(
        goal_positions=
                {reachy.r_arm.r_shoulder_pitch: 50,
                reachy.r_arm.r_shoulder_roll: -15,
                reachy.r_arm.r_arm_yaw: 0,
                reachy.r_arm.r_elbow_pitch: -100,
                reachy.r_arm.r_forearm_yaw: -15,
                reachy.r_arm.r_wrist_pitch: -60,
                reachy.r_arm.r_wrist_roll: 0},
            duration=1.0,
            interpolation_mode=InterpolationMode.MINIMUM_JERK
        )

In [ ]:
reachy.turn_off_smoothly('reachy')

In [ ]:
trajectories = [] #on créer une nouvelle liste trajectoire

start = time.time() #seconde passées depuis epoch
while (time.time() - start) < record_duration:
    #on optient les positions actuelles de toutes les jointures 
    current_point = [joint.present_position for joint in recorded_joints]
    #print(current_point)
    #on ajoute les positions à la liste trajectoire 
    trajectories.append(current_point)

    time.sleep(1 / sampling_frequency)

traj = {}
for joint,val in zip(recorded_joints,np.array(trajectories).T):
    joint = joint.name
    print(joint)
    if 'neck' in joint : 
        fullName = 'reachy.head.'+joint
    elif joint.startswith('r_') : 
        fullName = 'reachy.r_arm.'+joint
    elif joint.startswith('l_') : 
        fullName = 'reachy.l_arm.'+joint
    traj[fullName]=val

print(traj)

Record simple position : 

In [ ]:
trajectories = [] #on créer une nouvelle liste trajectoire

start = time.time() #seconde passées depuis epoch
#on optient les positions actuelles de toutes les jointures 
current_point = [joint.present_position for joint in recorded_joints]
#print(current_point)
#on ajoute les positions à la liste trajectoire 
trajectories.append(current_point)

traj = {}
for joint,val in zip(recorded_joints,np.array(trajectories).T):
    joint = joint.name
    print(joint)
    if 'neck' in joint : 
        fullName = 'reachy.head.'+joint
    elif joint.startswith('r_') : 
        fullName = 'reachy.r_arm.'+joint
    elif joint.startswith('l_') : 
        fullName = 'reachy.l_arm.'+joint
    traj[fullName]=val

print(traj)

Save as a .npz format : 

In [ ]:
np.savez('/home/reachy/dev/tictactoe2021/reachy_tictactoe/moves-2021_nemo/put_1',move=traj)